In [1]:
import torch
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=True):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        # GRU layer
        self.gru = nn.GRU(input_size, self.hidden_size, num_layers = 1)
        
        # Fully connected layer to map the hidden state to output
        self.fc = nn.Linear(self.hidden_size, input_size)
        
    def forward(self, x):
        batch_size = x.size(1)
        x.size(2)
        hidden_state = torch.ones(1,batch_size, self.hidden_size)#num_layer, batch_size, hidden_dim
        iters = 10
        print(x.shape)
        out,h = self.gru(x, hidden_state)
        print(out.shape)
        print(self.fc(out[:, -1, :]).shape)
        for t in range(iters):
            out, hidden_state = self.gru(x, hidden_state)
            x = self.fc(out[:, -1, :])
            x = x.reshape(-1,1,input_size) 
        return x

In [ ]:

class SparseNeuralNetwork(nn.Module):
    def __init__(self, in_dim, out_dim, h = [8,4]):
        super(SparseNeuralNetwork, self).__init__()
        h = [1] + h
        self.univariate_nn = nn.Sequential()
        layers = []
        self.masks = []
        for layer in range(1,len(h)):
            layers.append(nn.Linear(h[layer -1] * in_dim, h[layer] * in_dim))
            layers.append(nn.ReLU())
            self.masks.append(self.hidden_sparistiy_masks(h[layer] * in_dim, h[layer -1] * in_dim, h[layer - 1],h[layer]))
        self.univariate_nn = nn.Sequential(*layers)
        print(self.univariate_nn)
        self.multiply_weight_masks()
        self.fc2 = nn.Linear(h[-1] * in_dim, out_dim)

    def multiply_weight_masks(self):
        with torch.no_grad():
            for i in range(0,len(self.univariate_nn),2):
                self.univariate_nn[i].weight.mul_(self.masks[i // 2])

    def multiply_grad_masks(self):
        with torch.no_grad():
            for i in range(0,len(self.univariate_nn),2):
                self.univariate_nn[i].weight.grad.mul_(self.masks[i // 2])

    def hidden_sparistiy_masks(self, out_dim=50, in_dim=5, input_neurons=1, output_neurons=10):
        mask = torch.zeros(out_dim, in_dim)
        for i in range(0,in_dim):
            mask[i*output_neurons:output_neurons*(i + 1) , i*input_neurons:(i + 1)*input_neurons] = 1
        mask = torch.vstack([mask for i in range(3)])
        return mask

    def forward(self, x):
        hidden = self.univariate_nn(x)
        output = self.fc2(hidden)
        return output